In [1]:
import requests
import pandas as pd
from datetime import datetime
import json
import urllib.request

# Function to clean dates
def clean_dates(date_str):
    try:
        return datetime.strptime(date_str, "%Y-%m-%d").date()
    except ValueError:
        return None

# Desired game types
game_types = ['blitz']  # Example: game types to filter by

# Chess.com API to fetch game archives
chess_username = 'owej9023'  # Replace with the actual username
api_url = f"https://api.chess.com/pub/player/{chess_username}/games/archives"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Fetching the archives
response = requests.get(api_url, headers=headers)

# Check if the request was successful
if response.status_code != 200:
    print(f"Error fetching data. Status code: {response.status_code}")
    print(f"Response content: {response.text}")
else:
    archives = response.json()
    if 'archives' not in archives:
        print("No archives found or unexpected response structure.")
    else:
        print("Archives fetched successfully.")

# Initialize a list to store combined game data
combined_game_data = []

# Iterate through each archive URL and fetch the games
for archive_url in reversed(archives['archives']):
    with urllib.request.urlopen(archive_url) as url:
        archive_game = json.load(url)
        
        # Check if the expected 'games' key exists
        if 'games' in archive_game:
            combined_game_data.extend(archive_game['games'])  # Add the games to the combined list
        else:
            print(f"Warning: No 'games' found in archive {archive_url}")

# After accumulating all game data, normalize and convert to DataFrame
game_df = pd.json_normalize(combined_game_data)


# Initialize an empty list to store filtered games
filtered_lst = []


# Iterate through each row in the DataFrame
for index, game in game_df.iterrows():
    #print(game)
    # Apply the filter conditions
    if game['time_class'].lower() in ['blitz', 'rapid', 'bullet'] and \
       game['rules'] == 'chess' and \
       game['rated']:
        # Add the game to the filtered list if it meets the conditions
        filtered_lst.append(game.to_dict())  # Convert the game row to a dictionary
# Create a DataFrame from the filtered games list
filtered_df = pd.json_normalize(filtered_lst)

chess_username = "owej9023"

# Add current_elo and opponent_elo columns to the DataFrame
filtered_df['current_elo'] = None
filtered_df['opponent_elo'] = None

# Assign values for games where the user plays as white
filtered_df.loc[
    filtered_df['white.username'] == chess_username, 
    ['current_elo', 'opponent_elo']
] = filtered_df.loc[
    filtered_df['white.username'] == chess_username, 
    ['white.rating', 'black.rating']
].values

# Assign values for games where the user plays as black
filtered_df.loc[
    filtered_df['black.username'] == chess_username, 
    ['current_elo', 'opponent_elo']
] = filtered_df.loc[
    filtered_df['black.username'] == chess_username, 
    ['black.rating', 'white.rating']
].values
filtered_data = filtered_df
for value in filtered_data['']
print(filtered_data)

Archives fetched successfully.
                                                url  \
0      https://www.chess.com/game/live/129497550571   
1      https://www.chess.com/game/live/129528141897   
2      https://www.chess.com/game/live/129528177563   
3      https://www.chess.com/game/live/129528193199   
4      https://www.chess.com/game/live/129528760385   
...                                             ...   
13112    https://www.chess.com/game/live/5922245430   
13113    https://www.chess.com/game/live/6049736009   
13114    https://www.chess.com/game/live/6050243208   
13115    https://www.chess.com/game/live/6050422007   
13116    https://www.chess.com/game/live/6050505823   

                                                     pgn time_control  \
0      [Event "Live Chess"]\n[Site "Chess.com"]\n[Dat...           60   
1      [Event "Live Chess"]\n[Site "Chess.com"]\n[Dat...          180   
2      [Event "Live Chess"]\n[Site "Chess.com"]\n[Dat...          180   
3      [Event "L

In [2]:
import re
def extract_game_data(pgn_str):
    """
    Extract relevant game data (metadata, moves, and timestamps) from a PGN string.
    
    :param pgn_str: A string containing a PGN game.
    :return: A dictionary with extracted game data.
    """
    # Extract game headers (metadata)
    header_pattern = r'\[([A-Za-z]+)\s+"([^"]+)"\]'
    headers = re.findall(header_pattern, pgn_str)
    game_info = {header[0]: header[1] for header in headers}
    
    # Ensure the 'Termination' field is extracted (if present in the header)
    if "Termination" not in game_info:
        game_info["Termination"] = ""

    # Extract player ratings, defaulting to empty string if not found
    game_info["WhiteElo"] = game_info.get("WhiteElo", "")
    game_info["BlackElo"] = game_info.get("BlackElo", "")
    
    # Extract moves and timestamps using the provided regex patterns
    def extract_moves_and_timestamps(pgn):
        # Regex patterns for moves and timestamps
        move_pattern = r"(\d+\.\s(?:O-O(?:-O)?|\w+)(?:=[QRBN])?|\d+\.\.\.\s(?:O-O(?:-O)?|\w+)(?:=[QRBN])?)"
        timestamp_pattern = r"\[%clk\s([\d:.]+)\]"

        # Extract moves and timestamps
        moves = re.findall(move_pattern, pgn)
        timestamps = re.findall(timestamp_pattern, pgn)

        # Check for mismatched lengths
        if len(moves) != len(timestamps):
            print("Warning: Moves and timestamps lengths do not match!")
        
        return {"moves": moves, "timestamps": timestamps}
    
    # Get moves and timestamps
    move_data = extract_moves_and_timestamps(pgn_str)
    
    # Separate columns for moves and timestamps
    game_info["Moves"] = move_data["moves"]
    game_info["Timestamps"] = move_data["timestamps"]
    
    return game_info

# Initialize empty lists to store each value
value1_list = []
value2_list = []

# Iterate over each value in the filtered_data['pgn'] and extract data
for value in filtered_data['pgn']:
    # Assuming x is a dictionary with keys like 'Moves' and 'Timestamps'
    x = extract_game_data(str(value))
    
    # Ensure that x contains 'Moves' and 'Timestamps' before processing
    if 'Moves' in x and 'Timestamps' in x:
        # Extract values for 'Moves' and 'Timestamps'
        value1_list.append(x['Moves'])
        value2_list.append(x['Timestamps'])
    else:
        # Append None if keys are not found
        value1_list.append(None)
        value2_list.append(None)

# Add the extracted values as new columns in the DataFrame
filtered_data['Moves'] = value1_list
filtered_data['Timestamps'] = value2_list



In [3]:
count = 0
for x in filtered_data['Moves']:
    if len(x) != len(filtered_data['Timestamps'][count]):
        print(count)
        count+=1
    else:
        count +=1
        #print('pass' + str(count))
        


In [4]:

import chess
import chess.engine
import pandas as pd

def analyze_moves_with_stockfish(moves, stockfish_path):
    """
    Analyze chess moves using Stockfish and return the evaluation for each move.
    
    Args:
        moves (list): List of moves in SAN (Standard Algebraic Notation), e.g., ['1. d4', '1... d5', ...]
        stockfish_path (str): Path to the Stockfish engine binary.
    
    Returns:
        list: List of evaluation scores after each move.
    """
    
    # Start the Stockfish engine
    with chess.engine.SimpleEngine.popen_uci(stockfish_path) as engine:
        # Initialize a list to store scores
        scores = []

        # Create a new chess board
        board = chess.Board()

        # Loop over each move in the list
        for move in moves:
            # Strip the move number and ellipsis (e.g., "1. d4" -> "d4", "1... d5" -> "d5")
            move_san = move.split(" ")[-1].strip()  # Extract just the move part (e.g., "d4", "d5")
            #print(move)
            if not move_san:
                continue  # Skip empty moves (if any)

            try:
                # Make the move on the board
                board.push_san(move_san)
                # Evaluate the position after the move
                info = engine.analyse(board, chess.engine.Limit(depth=5))
                score = info["score"].relative.score(mate_score=5000)  # Use a high value for mate
                scores.append(score)
            except (ValueError, chess.IllegalMoveError):
                # If the move is invalid or illegal, append None and skip this move
                scores.append(None)

    return scores



In [5]:
stockfish_path = r"C:\Users\riann\Downloads\stockfish\stockfish-windows-x86-64-avx2.exe"

In [6]:
filtered_data_test = filtered_data.iloc[0:3]
test = []

# Iterate over each row in filtered_data_test and analyze the moves
for index, row in filtered_data_test.iterrows():
    moves = row["Moves"]
    
    analysis = analyze_moves_with_stockfish(moves, stockfish_path)  # Pass individual 'moves'
    
    test.append(analysis)

# Assign the analysis results to a new column
filtered_data_test['Stockfish Analysis'] = test

# Display the resulting DataFrame
filtered_data_test



FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:
from tqdm import tqdm  # Import tqdm for progress bar

filtered_data_test = filtered_data.iloc[0:3]
test = []

# Use tqdm to wrap your loop for progress bar
for index, row in tqdm(filtered_data.iterrows(), total=filtered_data.shape[0], desc="Analyzing Moves"):
    moves = row["Moves"]
    
    # Analyze the moves with Stockfish
    analysis = analyze_moves_with_stockfish(moves, stockfish_path)  # Pass individual 'moves'
    
    test.append(analysis)

# Assign the analysis results to a new column
filtered_data['Stockfish Analysis'] = test

# Display the resulting DataFrame
filtered_data




In [ ]:
#filtered_data.to_csv('filtered_data_with_analysis.csv', index=False)
#need a way to turn all these lists into individual values
import pandas as pd
import dask.dataframe as dd
import ast

filtered_data = pd.read_csv("filtered_data_with_analysis.csv") 
filtered_data['Moves'] = filtered_data['Moves'].apply(ast.literal_eval)
filtered_data_exploded = filtered_data.explode('Moves', ignore_index=True)
filtered_data_exploded.to_csv('filtered_data_with_analysis.csv', index=False)


In [ ]:
import pandas as pd
import os
import ast

# Read the data
df = pd.read_csv('filtered_data_with_analysis.csv')

# Set output directory
output_dir = 'C:/Users/19258/Downloads/parquet'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Ensure all relevant columns contain actual lists
for col in ['Moves', 'Timestamps', 'Stockfish Analysis']:
    df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Initialize chunk index for file naming
chunk_index = 0

# Process each row to minimize memory usage
for idx, row in df.iterrows():
    # Access the values in the current row directly
    moves_list = row['Moves']
    timestamps_list = row['Timestamps']
    stockfish_list = row['Stockfish Analysis']
    
    # Debug: Print the lengths of the lists
    moves_len = len(moves_list)
    timestamps_len = len(timestamps_list)
    stockfish_len = len(stockfish_list)
    
    print(f"Row {idx} - Moves length: {moves_len}, Timestamps length: {timestamps_len}, Stockfish Analysis length: {stockfish_len}")

    # Ensure all columns have equal length
    if not (moves_len == timestamps_len == stockfish_len):
        print(f"Skipping row {idx} because column lengths don't match!")
        continue  # Skip this row if lengths don't match

    # Create a DataFrame where each column is exploded separately
    exploded_df = pd.DataFrame({
        'Moves': moves_list,
        'Timestamps': timestamps_list,
        'Stockfish Analysis': stockfish_list
    })

    # Check how many rows after explosion
    print(f"After exploding, row {idx} has {len(exploded_df)} rows")

    # Save exploded data to Parquet incrementally using pandas
    file_name = f"chunk_{chunk_index + 1}_exploded.parquet"
    file_path = os.path.join(output_dir, file_name)

    # Writing to Parquet using pandas and fastparquet
    exploded_df.to_parquet(file_path, engine='fastparquet')

    print(f"Saved exploded data to {file_name}")
    chunk_index += 1

In [ ]:

import pandas as pd
import os
import ast

# Read the data
df = pd.read_csv('filtered_data_with_analysis.csv')

# Set output directory
output_dir = 'C:/Users/19258/Downloads/parquet'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Ensure all relevant columns contain actual lists
for col in ['Moves', 'Timestamps', 'Stockfish Analysis']:
    df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Initialize chunk index for file naming
chunk_index = 0

# Process each row to minimize memory usage
for idx, row in df.iterrows():
    # Access the values in the current row directly
    moves_list = row['Moves']
    timestamps_list = row['Timestamps']
    stockfish_list = row['Stockfish Analysis']
    
    # Debug: Print the lengths of the lists
    #moves_len = len(moves_list)
    #timestamps_len = len(timestamps_list)
    #stockfish_len = len(stockfish_list)
    
    #print(f"Row {idx} - Moves length: {moves_len}, Timestamps length: {timestamps_len}, Stockfish Analysis length: {stockfish_len}")

    # Ensure all columns have equal length
    if not (moves_len == timestamps_len == stockfish_len):
        print(f"Skipping row {idx} because column lengths don't match!")
        continue  # Skip this row if lengths don't match

    # Create a DataFrame for the other columns (non-exploded columns)
    other_columns = row.drop(['Moves', 'Timestamps', 'Stockfish Analysis'])

    # Create a DataFrame where the exploded columns are aligned with other columns
    exploded_df = pd.DataFrame({
        'Moves': moves_list,
        'Timestamps': timestamps_list,
        'Stockfish Analysis': stockfish_list
    })
    
    # Now, repeat the other columns (non-exploded) to match the exploded rows
    for col in other_columns.index:
        exploded_df[col] = other_columns[col]

    # Check how many rows after explosion
    #print(f"After exploding, row {idx} has {len(exploded_df)} rows")

    # Save exploded data to Parquet incrementally using pandas
    file_name = f"chunk_{chunk_index + 1}_exploded.parquet"
    file_path = os.path.join(output_dir, file_name)

    # Writing to Parquet using pandas and fastparquet
    exploded_df.to_parquet(file_path, engine='fastparquet')

    #print(f"Saved exploded data to {file_name}")
    chunk_index += 1


In [ ]:
import pyarrow.parquet as pq
import pandas as pd
import os

# Define the directory where the Parquet files are saved
output_dir = 'C:/Users/19258/Downloads/parquet'


# List all Parquet files in the output directory
parquet_files = [f for f in os.listdir(output_dir) if f.endswith('.parquet')]

# List to store DataFrame chunks
df_list = []

# Load each Parquet file and append it to the list
for file in parquet_files:
    file_path = os.path.join(output_dir, file)
    
    # Read the Parquet file into a PyArrow Table
    table = pq.read_table(file_path)
    
    # Convert the PyArrow Table to a pandas DataFrame
    df_chunk = table.to_pandas()
    
    # Append the chunk to the list
    df_list.append(df_chunk)

# Concatenate all the chunks into a single DataFrame
df_full = pd.concat(df_list, ignore_index=True)
#print(len(df_full))

df_full

Fully Filter the Data

In [ ]:
def time_to_seconds(time_str):
    # Split the time string into hours, minutes, and seconds
    parts = time_str.split(':')
    hours = int(parts[0])  # Always present
    minutes = int(parts[1])  # Always present
    seconds = parts[2]  # Can have decimal points or not
    
    # If seconds contains a decimal, split it into seconds and tenths
    if '.' in seconds:
        seconds, tenths = map(int, seconds.split('.'))
    else:
        seconds = int(seconds)
        tenths = 0  # Default to 0 if no tenths are present

    # Calculate total seconds
    total_seconds = (hours * 3600) + (minutes * 60) + seconds + (tenths / 10)
    return total_seconds

# Apply the time_to_seconds function to the 'Timestamps' column
result_df['Timestamps'] = result_df['Timestamps'].apply(time_to_seconds)


In [ ]:
#all the data is processed except for strings like 180+2 or 600+30


from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
# Ensure all target columns are numeric
result_df["current_elo"] = pd.to_numeric(result_df["current_elo"], errors="coerce")
result_df["Moves"] = pd.to_numeric(result_df["Moves"], errors="coerce")
result_df["time_class"] = pd.to_numeric(result_df["time_class"], errors="coerce")

# Handle NaN values (e.g., replace with 0 or mean of the column)
result_df.fillna(0, inplace=True)

# Re-define the target
y = result_df[["current_elo", "Moves"]].to_numpy()

# Encode "Move" column to numeric values
label_encoder = LabelEncoder()
result_df['Moves'] = label_encoder.fit_transform(result_df['Moves'].astype(str))  # Convert moves to numeric

# Define features
features = ["time_class","opponent_elo", "current_elo", 
            "Timestamps", "Moves", "Stockfish Analysis", "time_control"]




In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Define a function to safely evaluate expressions
def safe_eval(value):
    try:
        return float(eval(value))  # Evaluate the expression and convert to float
    except:
        return np.nan  # Return NaN if the evaluation fails

# Apply the function to the 'features' column
result_df[features] = result_df[features].applymap(safe_eval)

# Now normalize the features
scaler = MinMaxScaler()
X = scaler.fit_transform(result_df[features].to_numpy())  # Normalized feature matrix

# Define multi-output target
y = result_df[["current_elo", "Moves"]].to_numpy()  # Multi-output target

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Output shapes for debugging
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

In [ ]:
#Testing the model
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


import torch
import torch.nn as nn
from torch.optim import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset

# Device setup (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create DataLoader for efficient batching
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

import torch.nn as nn
import torch.nn.functional as F

class ChessNN(nn.Module):
    def __init__(self, input_size):
        super(ChessNN, self).__init__()
        # Define the layers
        self.fc1 = nn.Linear(input_size, 128)  # First fully connected layer
        self.fc2 = nn.Linear(128, 64)         # Second fully connected layer
        self.fc3 = nn.Linear(64, 2)           # Output layer (2 predictions)

    def forward(self, x):
        # Forward pass
        x = F.relu(self.fc1(x))  # Apply ReLU activation
        x = F.relu(self.fc2(x))
        x = self.fc3(x)          # Output layer (no activation for regression)
        return x



# Model, optimizer, and loss function
model = ChessNN(X_train.shape[1]).to(device)
optimizer = Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()  # For regression

# Training loop
epochs = 20
for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    for batch_x, batch_y in train_loader:
        # Move batch to GPU
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        optimizer.zero_grad()
        output = model(batch_x)
        loss = loss_fn(output, batch_y)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {epoch_loss / len(train_loader)}")

# Evaluate the model
model.eval()
with torch.no_grad():
    output = model(X_test_tensor).cpu().numpy()  # Move to CPU for numpy conversion
    pred_current_elo = output[:, 0]
    pred_time_control = output[:, 1]

    actual_current_elo = y_test[:, 0]
    actual_time_control = y_test[:, 1]

    # Metrics for current_elo
    mae_elo = mean_absolute_error(actual_current_elo, pred_current_elo)
    rmse_elo = mean_squared_error(actual_current_elo, pred_current_elo, squared=False)
    r2_elo = r2_score(actual_current_elo, pred_current_elo)
    print(f"Current Elo MAE: {mae_elo}, RMSE: {rmse_elo}, R^2: {r2_elo}")

    # Metrics for time_control
    mae_tc = mean_absolute_error(actual_time_control, pred_time_control)
    rmse_tc = mean_squared_error(actual_time_control, pred_time_control, squared=False)
    r2_tc = r2_score(actual_time_control, pred_time_control)
    print(f"Move MAE: {mae_tc}, RMSE: {rmse_tc}, R^2: {r2_tc}")

    # Combine results
    test_game_numbers = result_df.loc[X_test[:, -1].astype(int), 'game_number']
    results_df = pd.DataFrame({
        "Game_Number": test_game_numbers,
        "Predicted_Current_Elo": pred_current_elo,
        "Actual_Current_Elo": actual_current_elo,
        "Predicted Move": pred_time_control,
        "Actual_move": actual_time_control
    }).drop_duplicates(subset=["Game_Number"])

    print(results_df)

# 